<a href="https://colab.research.google.com/github/ilkersigirci/ML-with-Colab/blob/master/formSubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# using numerical indexes -> iloc
# using labels as indexes -> loc
# df.iloc[0:3]     # for getting rows
# df.loc[[0,1,2]]  # for getting rows
# df[["id", "details"]]  # for getting columns

# if conditions
# df[ (df["id"] > '70') & (df["id"] < '170') ]

# aggregate -> Quantity of sales over each country & Region
# df.groupby(["Country", "Region"])["Quantity"].sum()

#replace
# df = df.replace(np.nan, 0, regex=True)  # replace nan with 0

# str to int
# df['DataFrame Column'] = pd.to_numeric(df['DataFrame Column'], errors='coerce')

# to str
# df[column_name] = df[column_name].apply(str)
# df.applymap(str) -> whole frame

In [0]:
import os
import pprint
import tensorflow as tf
tf.reset_default_graph()
import pandas as pd
import numpy as np
import time
from google.colab import drive
drive.mount('/content/drive/')
#os.chdir("/content")

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
!mkdir mydata
!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/original.csv  mydata/
#!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/preprocessed.csv  mydata/

In [0]:
########################### UNEDITED CSV #########################################

!cp drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/support_forum_questions.csv  mydata/

dfPath = 'mydata/support_forum_questions.csv'
original = pd.read_csv(dfPath, sep='|')
original.drop(["added", "login"], axis = 1,inplace=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
pd.set_option('display.max_colwidth', -1)
dfPath = 'mydata/original.csv'
#dfPath = 'mydata/preprocessed.csv'
df = pd.read_csv(dfPath, sep='|',index_col=0)

#axis 0 => axis='rows' / axis='index'
#axis 1 => axis='columns'
#df.drop(["added", "login"], axis = 1,inplace=True)
df.shape

(219340, 4)

In [0]:
df

In [0]:
#df = df[pd.notnull(df['id'])]

df["id"].isnull().sum() # 2 null

0

In [0]:
df.dropna(axis = 0, how='any',inplace=True)
df.reset_index(drop=True,inplace=True)
df["id"] = df["id"].apply(str)
df.shape

(219598, 4)

In [0]:
count = 0
for index in df["id"].values:
    if(type(index) != str):
        #print(index)
        count = count +1
    
print (count)

0


In [0]:
#import re
#p = re.compile(r'<.*?>')
#df["question"] = [p.sub(' ', x) for x in df["question"]]
#df["details"] = [p.sub(' ', x) for x in df["details"]]

#Remove punctuations
#df["question"] = [re.sub('[^a-zA-Z1-9]', ' ', x) for x in df["question"]]
#df["details"] = [re.sub('[^a-zA-Z1-9]', ' ', x) for x in df["details"]]
#remove tags
#df["question"] = [re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",x) for x in df["question"]]
#df["details"] = [re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",x) for x in df["details"]]
#remove multiple spaces
#df["question"] = [re.sub(" +"," ",x) for x in df["question"]]
#df["details"] = [re.sub(" +"," ",x) for x in df["details"]]

In [0]:
#cleaning text
import re
import nltk
from string import punctuation
from nltk.stem import SnowballStemmer

def pad_str(s):
    return ' '+s+' '
def pad_number(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)
def pad_pattern(pattern):
    matched_string = pattern.group(0)
    return pad_str(matched_string)

def clean(text):
    
    if pd.isnull(text):
        return ''
    
    # Empty question   
    if type(text) != str or text=='':
        return ''
    
    # Clean the text, with the option to stem words.
    # stops = set(stopwords.words("english"))
    
    ###########################################################################################
    ################################# SPECIFIC TO JOTFORM #####################################
    ###########################################################################################
    text = re.sub('<[^<]+?>', ' ',                  text, flags=re.IGNORECASE)  # remove html
    text = re.sub('/(\r\n)+|\r+|\n+|\t+/i', '' ,    text, flags=re.IGNORECASE)  # remove \r \n
    text = re.sub("nbsp", "",                       text, flags=re.IGNORECASE)  # remove nbsp
    text = ''.join([c for c in text if c not in punctuation]).lower()           # remove punc
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",     text, flags=re.IGNORECASE)  # remove tags
    text = re.sub("http\S*", "",                    text, flags=re.IGNORECASE)  # remove links
    text = re.sub("www\S*", "",                     text, flags=re.IGNORECASE)  # remove links
    text = re.sub("\S*@\S*\s?", "",                 text, flags=re.IGNORECASE)  # remove email
    text = re.sub("jo[a-z]*form", "jotform",        text, flags=re.IGNORECASE)  # fix1 jotform
    text = re.sub("jot[a-z]+", "jotform",           text, flags=re.IGNORECASE)  # fix2 jotform
    text = re.sub("jotform", "form",                text, flags=re.IGNORECASE)  # jotform to form
    text = re.sub("colour", "color",                text, flags=re.IGNORECASE)  # fix colour
    text = re.sub("authorizenet", "payment",        text, flags=re.IGNORECASE)
    text = re.sub("calender", "calendar",           text, flags=re.IGNORECASE)
    
    text = re.sub("autosuspended", "auto suspend",  text, flags=re.IGNORECASE)
    text = re.sub("autoresponses", "auto response", text, flags=re.IGNORECASE)
    text = re.sub("wordpresscom", "word press",     text, flags=re.IGNORECASE)
    text = re.sub("myforms", "my form",             text, flags=re.IGNORECASE)
    text = re.sub("onedrive", "cloud",              text, flags=re.IGNORECASE)
    text = re.sub("gmailcom", "email",              text, flags=re.IGNORECASE)
    text = re.sub("jo[a-z]*form", "form",           text, flags=re.IGNORECASE)
    text = re.sub("unrequire", "not require",       text, flags=re.IGNORECASE)
    text = re.sub("prepopulate[a-z]*", "populate",  text, flags=re.IGNORECASE)
    text = re.sub("sumbissions", "submission",      text, flags=re.IGNORECASE)
    text = re.sub("submissons", "submission",       text, flags=re.IGNORECASE)
    text = re.sub("mozilla\S*", "browser",          text, flags=re.IGNORECASE)
    text = re.sub("chrome\S*", "browser",           text, flags=re.IGNORECASE)
    
    
    ###########################################################################################
    ################################# SPECIFIC TO JOTFORM #####################################
    ###########################################################################################
    
    #Fix Negative
    text = re.sub("n't", " not ",                   text, flags=re.IGNORECASE)
    text = re.sub("isnt", "is not",                 text, flags=re.IGNORECASE)
    text = re.sub("arent", "are not",               text, flags=re.IGNORECASE)
    text = re.sub("dont", "do not",                 text, flags=re.IGNORECASE)
    text = re.sub("doesnt", "does not",             text, flags=re.IGNORECASE)
    text = re.sub("didnt", "did not",               text, flags=re.IGNORECASE)
    text = re.sub("shouldnt", "should not",         text, flags=re.IGNORECASE)
    text = re.sub("hasnt", "has not",               text, flags=re.IGNORECASE)
    text = re.sub("havent", "have not",             text, flags=re.IGNORECASE)
    text = re.sub("wont", "will not",               text, flags=re.IGNORECASE)
    text = re.sub("cant", "can not",                text, flags=re.IGNORECASE)
    text = re.sub("couldnt", "could not",           text, flags=re.IGNORECASE)
    
    text = re.sub("\'s", " ",                       text, flags=re.IGNORECASE)
    text = re.sub("whats", "what is",               text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ",                 text, flags=re.IGNORECASE)    
    text = re.sub("i'm", "i am",                    text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ",                  text, flags=re.IGNORECASE)
    text = re.sub("\'d", " would ",                 text, flags=re.IGNORECASE)
    text = re.sub("\'ll", " will ",                 text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " eg ",                 text, flags=re.IGNORECASE)
    text = re.sub("e-mail", " email ",              text, flags=re.IGNORECASE)
    text = re.sub('\$', " dollar ",                 text, flags=re.IGNORECASE)
    text = re.sub('\%', " percent ",                text, flags=re.IGNORECASE)
    text = re.sub('\&', " and ",                    text, flags=re.IGNORECASE)
    text = re.sub("android","operating system",     text, flags=re.IGNORECASE)
    text = re.sub("ios", "operating system",        text, flags=re.IGNORECASE)
    text = re.sub('[0-9]+\.[0-9]+', "flt_number",   text, flags=re.IGNORECASE)  # replace the float numbers with flt_number    
    text = re.sub(" +"," ",                         text, flags=re.IGNORECASE)  # remove multiple space
    text = re.sub('[^\x00-\x7F]+', "",              text)                       # remove non-ascii characters
    
    
    #text = re.sub("(\d+)(kK)", " \g<1>000 ",        text, flags=re.IGNORECASE)
    #text = re.sub("\(s\)", " ",                     text, flags=re.IGNORECASE)
    #text = re.sub("[c-fC-F]\:\/", " disk ",         text, flags=re.IGNORECASE)
    #text = re.sub("(?<=[0-9])rs ", " rs ",          text, flags=re.IGNORECASE)
    #text = re.sub(" rs(?=[0-9])", " rs ",           text, flags=re.IGNORECASE)
    #text = re.sub(r" \0rs ", " rs ",                text, flags=re.IGNORECASE)    
    #text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii characters with non_ascii_word
    #text =re.sub('(?<=[0-9])\,(?=[0-9])',"",text, flags=re.IGNORECASE)         # remove comma between numbers
    #text = re.sub('[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\]', pad_pattern, text)        
    #text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    #text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)    
    # text = re.sub('[0-9]+', pad_number,  text, flags=re.IGNORECASE)           # all numbers should separate from words, this is too aggressive   

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [0]:
t1 = time.time()
df['question'] = df['question'].apply(clean)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 0.3 min
t1 = time.time()
df['details'] = df['details'].apply(clean)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 3 min

Calculation took  0.3677972912788391 min
Calculation took  3.275190814336141 min


In [0]:
######################################### Glove ############################################## 
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip -P mydata/
#!unzip mydata/glove.840B.300d.zip -d mydata/
#!cp drive/My\ Drive/Colab\ Notebooks/Dataset/glove.840B.300d.txt mydata/
#!touch mydata/glove_word2vec.txt
#_ = glove2word2vec(r'mydata/glove.840B.300d.txt', r"mydata/glove_word2vec.txt")

!cp  drive/My\ Drive/Colab\ Notebooks/Dataset/glove_word2vec.txt mydata/

In [0]:
from gensim.models import KeyedVectors
t1 = time.time()
glove_model = KeyedVectors.load_word2vec_format("mydata/glove_word2vec.txt", binary=False)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 11 min

In [0]:
glove_model.save("mydata/glove.model")

In [0]:
#noneng bulma vardi
from collections import Counter
c=Counter(noneng)
sorted_c = sorted(c.items(), key=lambda kv: kv[1],reverse=True)

In [0]:
def fix_words(text):
    text = re.sub("autosuspended", "auto suspend",  text, flags=re.IGNORECASE)
    text = re.sub("autoresponses", "auto response", text, flags=re.IGNORECASE)
    text = re.sub("wordpresscom", "word press",     text, flags=re.IGNORECASE)
    text = re.sub("myforms", "my form",             text, flags=re.IGNORECASE)
    text = re.sub("onedrive", "cloud",              text, flags=re.IGNORECASE)
    text = re.sub("gmailcom", "email",              text, flags=re.IGNORECASE)
    text = re.sub("jo[a-z]*form", "form",           text, flags=re.IGNORECASE)
    text = re.sub("unrequire", "not require",       text, flags=re.IGNORECASE)
    text = re.sub("prepopulate[a-z]*", "populate",  text, flags=re.IGNORECASE)
    text = re.sub("sumbissions", "submission",      text, flags=re.IGNORECASE)
    text = re.sub("submissons", "submission",       text, flags=re.IGNORECASE)
    text = re.sub("mozilla\S*", "browser",          text, flags=re.IGNORECASE)
    text = re.sub("chrome\S*", "browser",           text, flags=re.IGNORECASE)
    return text

In [0]:
t1 = time.time()
df['question'] = df['question'].apply(fix_words)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 0.06 min
t1 = time.time()
df['details'] = df['details'].apply(fix_words)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 0.2 min

Calculation took  0.07739330530166626 min
Calculation took  0.23662294944127402 min


In [0]:
import nltk
import re
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = stopwords.words('english')

def preprocess(text):
    
    ######################### NON-ENGLISH REMOVAL ################################

    text = " ".join(word for word in nltk.wordpunct_tokenize(text) if word.lower() in glove_model.vocab and not word.isdigit())
   
    ############################### LEMMATIZE ####################################
    
    from nltk import word_tokenize, pos_tag
    from nltk.stem.wordnet import WordNetLemmatizer
    #nltk.download('wordnet')
    #nltk.download('punkt')
    #nltk.download('averaged_perceptron_tagger')
    
    #no tag
    #text  = ' '.join([ lemmatizer.lemmatize(word) for word in text.split() ])
    
    # n -> noun / v -> verb / a -> adjective / r -> adverb
    tokens = word_tokenize(text)
    convert_tag = lambda e: ('a' if e[0].lower() == 'j' else e[0].lower()) if e[0].lower() in ['n', 'r', 'v'] else 'n'    
    lemmatizer = WordNetLemmatizer()    
    text  = ' '.join([ lemmatizer.lemmatize(token, convert_tag(tag)) for token,tag in pos_tag(tokens) ])   
    
    return text

In [0]:
t1 = time.time()
#test_q = df['question'].apply(preprocess)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 3 min
t1 = time.time()
test_d = df['details'].apply(preprocess)
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took 11 min

Calculation took  3.933906555175781e-07 min
Calculation took  11.558749902248383 min


In [0]:
count = 0 #135 / 127
indexes = []
for index,sentence in enumerate(df['question']):
    if sentence == "":
        indexes.append(index)
        count = count + 1

indexes = np.array(indexes)
print(count)
#print(df["details"][df.index[indexes]])

In [0]:
df['question'] = test_q
df['details'] = test_d

In [0]:
#detect empty rows
df['question'] = df['question'].apply(lambda x: re.sub(" +"," ",x,flags=re.IGNORECASE))
df['details' ] = df['details' ].apply(lambda x: re.sub(" +"," ",x,flags=re.IGNORECASE))

In [0]:
# drop "" and " " rows
#df.drop(df[condition].index, axis=0, inplace=True)

#question -> 69 / 158
#details  -> 1  / 1340

df.drop(df[df['question'] == "" ].index, axis=0, inplace=True)
df.drop(df[df['question'] == " "].index, axis=0, inplace=True)
df.drop(df[df['details' ] == "" ].index, axis=0, inplace=True)
df.drop(df[df['details' ] == " "].index, axis=0, inplace=True)
df.reset_index(drop=True,inplace=True)

count = 0
for sentence in df['details']:
    if sentence == " ":
        count = count + 1
        
print(count)
print(df.shape) #219340

0
(219340, 4)


In [0]:
df['all'] = df['question'] +' '+ df['details']

In [0]:
df.head(20)

In [0]:
"""
df['less8'] = df['all'].apply(lambda x: " " if len(x.split()) <= 8 else "normal")
df.drop(df[df['less8'] == " "].index, axis=0, inplace=True)
df.reset_index(drop=True,inplace=True)
"""
# drop sentences less than 3 words
short_indexes = []
for index,sentence in enumerate(df['all']):
    if(len(sentence.split()) <= 3):
        short_indexes.append(index)

df.drop(df.index[short_indexes],inplace = True)
df.reset_index(drop=True,inplace=True)

count = 0        # normalde 5087
for sentence in df['all']:
    if(len(sentence.split()) <= 8):
        count = count + 1

print(count)
print(df.shape) #222518

4337
(226672, 4)


In [0]:
# drop non english rows  -> 6888
!pip install langdetect
from langdetect import detect

t1 = time.time()

non_english = []
sen = []
for index,sentence in enumerate(df['all']):
    try:
        if detect(sentence) != "en":
            sen.append(sentence)
            non_english.append(index)
    except:
        sen.append(sentence)
        non_english.append(index)
        
t2 = time.time()
print("Calculation took ",(t2-t1)/60, "min")  # Took ? min 

df.drop(df.index[non_english],inplace = True)
df.reset_index(drop=True,inplace=True)

In [0]:
#remove rows that contains just stop words  -> 8

test_all = df['all'].apply( lambda x: " ".join(word for word in x.split() if word not in stop_words) )
count = 0
indexes = []
for index,sentence in enumerate(test_all):
    if sentence == "":
        indexes.append(index)
        count = count + 1

indexes = np.array(indexes)
print(count)
print(df["all"][df.index[indexes]])

df.drop(df.index[indexes],inplace = True)
df.reset_index(drop=True,inplace=True)

In [0]:
df.to_csv("mydata/original.csv", sep='|')
!cp mydata/original.csv drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/  

In [0]:
#remove stop_words
df['all'] = df['all'].apply( lambda x: " ".join(word for word in x.split() if word not in stop_words) )

df.to_csv("mydata/preprocessed.csv", sep='|')
!cp mydata/preprocessed.csv drive/My\ Drive/Colab\ Notebooks/Dataset/jotform/  

In [0]:
################################################################################################
##################################### End of Preprocess ########################################
################################################################################################

In [0]:
questions = df['question']
questions = np.array(questions)

In [0]:
################################################################################################
################################ Word Mover's Distance (WMD) ###################################
################################################################################################

In [0]:
#Word Mover's Distance (WMD)

#Copy GoogleNews-vectors-negative300.bin.gz
!cp  drive/My\ Drive/Colab\ Notebooks/Dataset/quora/GoogleNews-vectors-negative300.bin.gz mydata/

import gensim 
googlenews_corpus_path = 'mydata/GoogleNews-vectors-negative300.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(googlenews_corpus_path, binary=True)
model.init_sims(replace=True) # normalize

In [0]:
# Train my word2vec model

from gensim.models import Word2Vec
# split sentences

#test = df.question.apply(lambda x: len(str(x)))
splitted = []
for sentence in questions:
    if type(sentence) == str:
        splitted.append(sentence.lower().split())

splitted = np.array(splitted)

# size      -> determines the dimensions of the word vectors
# window    -> number of forward and backward words used in training
# min_count -> number of minimum times a word needs to appear to be included in the training
my_model = Word2Vec(splitted, size = 10, window = 5, min_count = 1)

In [0]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def my_wmd(q1, q2,model_type):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return model_type.wmdistance(q1, q2)

#all_except_index = np.delete(a, ind, axis=0) 

my_question = "how do I export data to google drive"
distance = []
for question in questions:
    distance.append(my_wmd(my_question,question,my_model))

distance = np.array(distance)

In [0]:
distance

In [0]:
# Get the indices of the four min elements -> returns unsorted
min_indexes = np.argpartition(distance, 4)[:4]
#sort it
min_indexes = min_indexes[np.argsort(distance[min_indexes])]

print("my_question       -> {}".format(my_question))
print("similar questions -> {}".format(original_questions[min_indexes]))

In [0]:
count = 0
for sentence in splitted:
    if(len(sentence) > 3):
        count = count +1

print(count)

for sentence in splitted:
    for word in sentence:
        if word == "jotform":
            count = count + 1
print(count)

In [0]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_words = stopwords.words('english')

#remove stopwords

#for i in range(len(splitted)):
#    splitted[i] = [word for word in splitted ]

count  = 0
nonenglish = []
for sentence in splitted:
    for word in sentence:
        if word not in model.vocab and word not in stop_words and word != "jotform":
            count = count + 1
            nonenglish.append(word)
            

print(count)
print(nonenglish)

In [0]:
# Training the model with list of sentences (with 4 CPU cores)
# model.train(sentences, workers=4)

#add word to GoogleNews model

print('jotform' in model.vocab)

# add jotform
model.add('jotform',model.get_vector('company'),replace=False)

In [0]:
################################################################################################
############################ Latent Dirichlet Allocation (LDA) #################################
################################################################################################

In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
nltk.download('wordnet')

In [0]:
dfPath = 'mydata/preprocessed.csv'
data = pd.read_csv(dfPath, sep='|') # ,dtype = str
data = data.fillna(" ")
data_questions = data[["question"]]
data_questions['index'] = data_questions.index
questions = data_questions

In [0]:
stemmer = SnowballStemmer('english')

# TODO : burayi duzenle sonradan
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
        
    return result

processed_questions = questions['question'].map(preprocess)

In [0]:
################ Bag of words on the dataset ############################

dictionary = gensim.corpora.Dictionary(processed_questions)

In [0]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [0]:
#less than 15 documents (absolute number) or
#more than 0.5 documents (fraction of total corpus size, not absolute number).
#keep only the first 100000 most frequent tokens.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_questions]
bow_corpus[4310]

In [0]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

In [0]:
####################### TF-IDF ############################

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

In [0]:
########### Running LDA using Bag of Words #################

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [0]:
########### Running LDA using TF-IDF #################

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [0]:
#Performance Evaluation 
print(processed_questions[4310])
print("##########################################################################\
################################################################################# ")
# Bag of Words
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

print("##########################################################################\
################################################################################# ")
print("##########################################################################\
################################################################################# ")
# Tf-idf
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

In [0]:
##### Testing Model ####

unseen_document = 'how do I export data to google drive'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

In [0]:
################################## OTHER METHODS #######################################

In [0]:
documents = questions
#change nan values to empty string
count = 0
for index, sentence in enumerate(documents):
    if type(sentence) == float:
        documents[index] = " "
        count = count + 1
        
print(count)